In [ ]:
!pip install google-play-scraper
!pip install nltk


In [ ]:
from google_play_scraper import reviews
import pandas as pd

# Mengambil review
reviews_data, _ = reviews(
    'com.bandainamcoent.dblegends_ww',
    lang='en',  # Bahasa Inggris
    country='us',  # Negara USA (United States of America)
    count=100  # Jumlah ulasan
)

# Membuat DataFrame
df = pd.DataFrame(reviews_data)
df.to_csv('dblegendsreviews.csv', index=False)
print(df.head())


                               reviewId          userName  \
0  3b6522c9-1d1c-4199-b6bb-3a1bc621cca3  Thomas Gillespie   
1  5887e276-4646-4869-aae8-c917941154b0       Vansh Singh   
2  d26fd9b9-d52b-49b5-89c2-9c0f3880fa9e          Kaiyoshi   
3  18e80f01-af5d-46db-9f78-37e037c2e9f1     Simran Sharma   
4  0e96427e-a948-4d58-8338-482bfa942bef     Anjana Mondal   

                                           userImage  \
0  https://play-lh.googleusercontent.com/a/ACg8oc...   
1  https://play-lh.googleusercontent.com/a/ACg8oc...   
2  https://play-lh.googleusercontent.com/a-/ALV-U...   
3  https://play-lh.googleusercontent.com/a-/ALV-U...   
4  https://play-lh.googleusercontent.com/a/ACg8oc...   

                                             content  score  thumbsUpCount  \
0  Toshi fix them rose banner is keep getting sha...      1              2   
1                                   I love this game      5              0   
2  its great game ever, but there is a problem, m...      4   

In [ ]:
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk

# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab') # This line is added to download the missing resource

# Membersihkan teks
def clean_text(text):
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"\d+", "", text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    text = text.lower()
    return text

# Menambah kolom teks bersih
df['cleaned_text'] = df['content'].apply(clean_text)

# Tokenisasi
df['tokens'] = df['cleaned_text'].apply(word_tokenize)

# Menghapus stopwords
stop_words = set(stopwords.words('english'))
df['tokens'] = df['tokens'].apply(lambda x: [word for word in x if word not in stop_words])

print(df[['cleaned_text', 'tokens']].head())

                                        cleaned_text  \
0  toshi fix them rose banner is keep getting sha...   
1                                   i love this game   
2  its great game ever but there is a problem muc...   
3                       it is the best animated game   
4  god geme ❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤...   

                                              tokens  
0  [toshi, fix, rose, banner, keep, getting, shaf...  
1                                       [love, game]  
2  [great, game, ever, problem, much, like, pvp, ...  
3                             [best, animated, game]  
4  [god, geme, ❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️...  


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df['cleaned_text']).toarray()

# Label sentimen berdasarkan skor
def label_sentiment(score):
    if score >= 4:
        return "Positive"
    elif score == 3:
        return "Neutral"
    else:
        return "Negative"

df['sentiment'] = df['score'].apply(label_sentiment)
print(df['sentiment'].value_counts())


sentiment
Positive    64
Negative    32
Neutral      4
Name: count, dtype: int64


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, df['sentiment'], test_size=0.2, random_state=42)

# Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)
y_pred_nb = nb_model.predict(X_test)

print("Naive Bayes Classification Report:")
print(classification_report(y_test, y_pred_nb))
print("Naive Bayes Accuracy:", accuracy_score(y_test, y_pred_nb))

# SVM
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)

print("SVM Classification Report:")
print(classification_report(y_test, y_pred_svm))
print("SVM Accuracy:", accuracy_score(y_test, y_pred_svm))


Naive Bayes Classification Report:
              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00         9
     Neutral       0.00      0.00      0.00         1
    Positive       0.50      1.00      0.67        10

    accuracy                           0.50        20
   macro avg       0.17      0.33      0.22        20
weighted avg       0.25      0.50      0.33        20

Naive Bayes Accuracy: 0.5
SVM Classification Report:
              precision    recall  f1-score   support

    Negative       0.80      0.44      0.57         9
     Neutral       0.00      0.00      0.00         1
    Positive       0.67      1.00      0.80        10

    accuracy                           0.70        20
   macro avg       0.49      0.48      0.46        20
weighted avg       0.69      0.70      0.66        20

SVM Accuracy: 0.7


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m